<a href="https://colab.research.google.com/github/Kanyi254/Insurance-quotation/blob/Brandon/notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective:

The goal of the project is to generate predictive insights for medical quota share treaties. We'll predict future premium payments or claims for policyholder groups (e.g., organizations like "Alliance") using historical data.



In [1]:
import pandas as pd

# Load datasets
premium_q3 = pd.read_excel('Medical quota share treaty  3rd Qtr 2020.xlsx')
premium_q4 = pd.read_excel('BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx')

claims_q3 = pd.read_excel('Medical quota share treaty  3rd Qtr 2020.xlsx', sheet_name='Claims Bordereaux')
claims_q4 = pd.read_excel('BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx', sheet_name='Claims Bordereaux')

# Basic cleaning: Drop unnamed columns, and missing values
premium_q3_clean = premium_q3.dropna(axis=1, how='all')  # Drop columns that are entirely NaN
premium_q4_clean = premium_q4.dropna(axis=1, how='all')

claims_q3_clean = claims_q3.dropna(axis=1, how='all')
claims_q4_clean = claims_q4.dropna(axis=1, how='all')

# Convert date columns to datetime
premium_q3_clean['Start Date of Cover'] = pd.to_datetime(premium_q3_clean['Start Date of Cover'], errors='coerce')
premium_q4_clean['Start Date of Cover'] = pd.to_datetime(premium_q4_clean['Start Date of Cover'], errors='coerce')

claims_q3_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q3_clean['Date of Claim/Treatment Date'], errors='coerce')
claims_q4_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q4_clean['Date of Claim/Treatment Date'], errors='coerce')

def fix_unnamed_columns(df):
  """Moves column names from 'Unnamed' columns to the index, and removes the original row."""
  new_columns = []
  rows_to_remove = []
  for i, col in enumerate(df.columns):
    if 'Unnamed' in col:
      if df[col].notna().any():
        new_columns.append(df[col][df[col].notna().idxmax()])
        rows_to_remove.append(df[col].notna().idxmax())
      else:
        new_columns.append(col)
    else:
      new_columns.append(col)
  df.columns = new_columns
  df = df.drop(rows_to_remove)
  return df

premium_q3_clean = fix_unnamed_columns(premium_q3_clean)
premium_q4_clean = fix_unnamed_columns(premium_q4_clean)
claims_q3_clean = fix_unnamed_columns(claims_q3_clean)
claims_q4_clean = fix_unnamed_columns(claims_q4_clean)

# View basic structure
print(premium_q3_clean.head())
print(claims_q4_clean.head())

                 Policy Holder ID  Principal beneficiary Dependants  \
1  AMBASSADEUR KWIZERA DIEUDONNE                     1.0          0   
2                  CHIRO BURUNDI                     8.0         11   
3    CENTRE BURUNDAIS D INTERNET                    39.0         99   
4                NTAHONDEREYE DEO                    1.0          0   
5            HELP CHANEL BURUNDI                    66.0        251   

   Total beneficiaries   Police ID Start Date of Cover End Date of Cover  \
1                  1.0  20000019.0          2020-07-01        2021-06-30   
2                 19.0  20000021.0          2020-07-17        2020-10-16   
3                138.0  20000022.0          2020-07-01        2021-06-30   
4                  1.0  20000023.0          2020-08-05        2021-08-04   
5                317.0  20000018.0          2020-07-01        2021-06-30   

            Number of  payment Installments allowed  \
1   1638558                                       1   
2   23

<ipython-input-1-eb18c91c303b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_q4_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q4_clean['Date of Claim/Treatment Date'], errors='coerce')


In [2]:
# prompt: combine claims together and premiums together

# Combine claims dataframes
claims_combined = pd.concat([claims_q3_clean, claims_q4_clean], ignore_index=True)

# Combine premium dataframes
premium_combined = pd.concat([premium_q3_clean, premium_q4_clean], ignore_index=True)

# Now you have two dataframes: claims_combined and premium_combined
# containing all claims and premium data respectively.

print(claims_combined.head())
print(premium_combined.head())

                     Policy Holder ID        Member ID Start Date of Cover  \
0                                 NaN              NaN                 NaT   
1  ALLIANCE BURUNDAISE CONTRE LE SIDA  NISUBIRE GERARD          2020-06-01   
2                                 NaN  NISUBIRE GERARD          2020-06-01   
3                                 NaN  NAHAYO CLAUDINE          2020-06-01   
4                                 NaN  NAHAYO CLAUDINE          2020-06-01   

  End Date of Cover Date of Claim/Treatment Date  \
0               NaT                          NaT   
1        2021-05-31                   2020-07-01   
2        2021-05-31                   2020-07-01   
3        2021-05-31                   2020-07-03   
4        2021-05-31                   2020-07-03   

  Date of Payment/Approval Date Amount Claimed Inpatient per family  \
0                           NaN            NaN                  NaN   
1                    2020-07-01          10000                  NaN   
2    

In [ ]:
# Group by Policy Holder ID to aggregate premium data
grouped_premiums_q4 = premium_q4_clean.groupby('Policy Holder ID').agg({
    'Premium Paid/Billed': 'sum',
    'Benefit Limit': 'sum'
}).reset_index()

# Group claims by Policy Holder ID
grouped_claims_q4 = claims_q4_clean.groupby('Policy Holder ID').agg({
    'Amount Paid': 'sum',
    'Benefit Limit': 'sum'
}).reset_index()

# View the grouped data
print(grouped_premiums_q4.head())
print(grouped_claims_q4.head())

                       Policy Holder ID  Premium Paid/Billed Benefit Limit
0  ALLIANCE BURUNDAISE CONTRE LE SIDA                10000.0       1200000
1           BICOR VIE ET CAPITALISATION           17907900.0       2400000
2                BURUNDI LEND AND LEASE              65972.0       1200000
3                  KCB BANK BURUNDI LTD            3160458.0      13000000
4              NDABARUSHIMANA DIEUDONNE            1374930.0       1500000
                     Policy Holder ID Amount Paid Benefit Limit
0             Ir SINDAYIHEBURA Lionel       10000       1200000
1  ALLIANCE BURUNDAISE CONTRE LE SIDA       24200       1200000
2                                BBIN       23000       1200000
3          BICOR ASSURANCES GENERALES       12000       1500000
4                           BICOR VIE           0       1080000


### Modelling

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd

# Example: Time series forecasting for a specific policyholder (e.g., 'ALLIANCE BURUNDAISE CONTRE LE SIDA')
def predict_premiums(data, policy_holder_group, forecast_periods=12):
    # Filter for the group
    data['Policy Holder ID'] = data['Policy Holder ID'].astype(str)
    group_data = data[data['Policy Holder ID'].str.contains(policy_holder_group, case=False, na=False)]

    # Use the 'Premium Paid/Billed' column for predictions
    premiums = group_data[['Start Date of Cover', 'Premium Paid/Billed']].dropna()

    # Convert index to datetime (Start Date of Cover)
    premiums['Start Date of Cover'] = pd.to_datetime(premiums['Start Date of Cover'])
    premiums.set_index('Start Date of Cover', inplace=True)

    # Resample to monthly data using 'ME'
    monthly_premiums = premiums.resample('ME').sum()

    # Check if we have enough data points (at least 12 months for forecasting)
    if len(monthly_premiums) < 12:
        print(f"Not enough data points for policy holder group '{policy_holder_group}' to perform forecasting.")
        return

    # Fit ARIMA model
    try:
        model = ARIMA(monthly_premiums, order=(5, 1, 0))
        model_fit = model.fit()

        # Forecast next 'forecast_periods' (e.g., 12 months)
        forecast, stderr, conf_int = model_fit.forecast(steps=forecast_periods)

        # Plot the forecast
        plt.plot(monthly_premiums, label='Actual Data')
        plt.plot(pd.date_range(start=monthly_premiums.index[-1], periods=forecast_periods, freq='M'), forecast, label='Forecast')
        plt.fill_between(pd.date_range(start=monthly_premiums.index[-1], periods=forecast_periods, freq='M'), conf_int[:, 0], conf_int[:, 1], color='lightgray', alpha=0.5)
        plt.legend()
        plt.title(f'Premium Forecast for {policy_holder_group}')
        plt.show()

    except Exception as e:
        print(f"ARIMA model error: {e}")

# Example: Predict for 'ALLIANCE BURUNDAISE CONTRE LE SIDA'
predict_premiums(premium_q4_clean, policy_holder_group='KCB BANK BURUNDI LTD', forecast_periods=12)


Not enough data points for policy holder group 'KCB BANK BURUNDI LTD' to perform forecasting.


In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
# # Objective:
#
# The goal of the project is to generate predictive insights for medical quota share treaties. We'll predict future premium payments or claims for policyholder groups (e.g., organizations like "Alliance") using historical data.
#
#

# Load datasets
premium_q3 = pd.read_excel('Medical quota share treaty  3rd Qtr 2020.xlsx')
premium_q4 = pd.read_excel('BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx')

claims_q3 = pd.read_excel('Medical quota share treaty  3rd Qtr 2020.xlsx', sheet_name='Claims Bordereaux')
claims_q4 = pd.read_excel('BICOR LIFE - Medical Quota share - 4th qtr 2020.xlsx', sheet_name='Claims Bordereaux')

# Basic cleaning: Drop unnamed columns, and missing values
premium_q3_clean = premium_q3.dropna(axis=1, how='all')  # Drop columns that are entirely NaN
premium_q4_clean = premium_q4.dropna(axis=1, how='all')

claims_q3_clean = claims_q3.dropna(axis=1, how='all')
claims_q4_clean = claims_q4.dropna(axis=1, how='all')

# Convert date columns to datetime
premium_q3_clean['Start Date of Cover'] = pd.to_datetime(premium_q3_clean['Start Date of Cover'], errors='coerce')
premium_q4_clean['Start Date of Cover'] = pd.to_datetime(premium_q4_clean['Start Date of Cover'], errors='coerce')

claims_q3_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q3_clean['Date of Claim/Treatment Date'], errors='coerce')
claims_q4_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q4_clean['Date of Claim/Treatment Date'], errors='coerce')

def fix_unnamed_columns(df):
  """Moves column names from 'Unnamed' columns to the index, and removes the original row."""
  new_columns = []
  rows_to_remove = []
  for i, col in enumerate(df.columns):
    if 'Unnamed' in col:
      if df[col].notna().any():
        new_columns.append(df[col][df[col].notna().idxmax()])
        rows_to_remove.append(df[col].notna().idxmax())
      else:
        new_columns.append(col)
    else:
      new_columns.append(col)
  df.columns = new_columns
  df = df.drop(rows_to_remove)
  return df

premium_q3_clean = fix_unnamed_columns(premium_q3_clean)
premium_q4_clean = fix_unnamed_columns(premium_q4_clean)
claims_q3_clean = fix_unnamed_columns(claims_q3_clean)
claims_q4_clean = fix_unnamed_columns(claims_q4_clean)

# View basic structure
print(premium_q3_clean.head())
print(claims_q4_clean.head())
# Group by Policy Holder ID to aggregate premium data
grouped_premiums_q4 = premium_q4_clean.groupby('Policy Holder ID').agg({
    'Premium Paid/Billed': 'sum',
    'Benefit Limit': 'sum'
}).reset_index()

# Group claims by Policy Holder ID
grouped_claims_q4 = claims_q4_clean.groupby('Policy Holder ID').agg({
    'Amount Paid': 'sum',
    'Benefit Limit': 'sum'
}).reset_index()

# View the grouped data
print(grouped_premiums_q4.head())
print(grouped_claims_q4.head())
# ### Modelling

# Example: Time series forecasting for a specific policyholder (e.g., 'ALLIANCE BURUNDAISE CONTRE LE SIDA')
def predict_premiums(data, policy_holder_group, forecast_periods=12):
    # Filter for the group
    data['Policy Holder ID'] = data['Policy Holder ID'].astype(str)
    group_data = data[data['Policy Holder ID'].str.contains(policy_holder_group, case=False, na=False)]

    # Use the 'Premium Paid/Billed' column for predictions
    premiums = group_data[['Start Date of Cover', 'Premium Paid/Billed']].dropna()

    # Convert index to datetime (Start Date of Cover)
    premiums['Start Date of Cover'] = pd.to_datetime(premiums['Start Date of Cover'])
    premiums.set_index('Start Date of Cover', inplace=True)

    # Resample to monthly data using 'ME'
    monthly_premiums = premiums.resample('M').sum()

    # Check if we have enough data points (at least 12 months for forecasting)
    if len(monthly_premiums) < 12:
        print(f"Not enough data points for policy holder group '{policy_holder_group}' to perform forecasting.")
        return

    # Fit ARIMA model
    try:
        model = ARIMA(monthly_premiums, order=(5, 1, 0))
        model_fit = model.fit()

        # Forecast next 'forecast_periods' (e.g., 12 months)
        forecast, stderr, conf_int = model_fit.forecast(steps=forecast_periods)

        # Plot the forecast
        plt.plot(monthly_premiums, label='Actual Data')
        plt.plot(pd.date_range(start=monthly_premiums.index[-1], periods=forecast_periods, freq='M'), forecast, label='Forecast')
        plt.fill_between(pd.date_range(start=monthly_premiums.index[-1], periods=forecast_periods, freq='M'), conf_int[:, 0], conf_int[:, 1], color='lightgray', alpha=0.5)
        plt.legend()
        plt.title(f'Premium Forecast for {policy_holder_group}')
        plt.show()

    except Exception as e:
        print(f"ARIMA model error: {e}")

# Example: Predict for 'ALLIANCE BURUNDAISE CONTRE LE SIDA'
predict_premiums(premium_q4_clean, policy_holder_group='KCB BANK BURUNDI LTD', forecast_periods=12)


                 Policy Holder ID  Principal beneficiary Dependants  \
1  AMBASSADEUR KWIZERA DIEUDONNE                     1.0          0   
2                  CHIRO BURUNDI                     8.0         11   
3    CENTRE BURUNDAIS D INTERNET                    39.0         99   
4                NTAHONDEREYE DEO                    1.0          0   
5            HELP CHANEL BURUNDI                    66.0        251   

   Total beneficiaries   Police ID Start Date of Cover End Date of Cover  \
1                  1.0  20000019.0          2020-07-01        2021-06-30   
2                 19.0  20000021.0          2020-07-17        2020-10-16   
3                138.0  20000022.0          2020-07-01        2021-06-30   
4                  1.0  20000023.0          2020-08-05        2021-08-04   
5                317.0  20000018.0          2020-07-01        2021-06-30   

            Number of  payment Installments allowed  \
1   1638558                                       1   
2   23

<ipython-input-60-edacda58b395>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  claims_q4_clean['Date of Claim/Treatment Date'] = pd.to_datetime(claims_q4_clean['Date of Claim/Treatment Date'], errors='coerce')
<ipython-input-60-edacda58b395>:89: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_premiums = premiums.resample('M').sum()
